#### 1 Categorical Data

In [14]:
import numpy as np; import pandas as pd
values = pd.Series(['apple', 'orange', 'apple',
                     'apple'] * 2)
pd.unique(values)
pd.value_counts(values)
#dimension table含有不同的value，以特定整数的形式存在dimension table里。
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
print(dim.take(values),'\n')

print('##################Categorical Type in pandas######################')
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit':fruits,
                  'basket_id': np.arange(N),
                  'count': np.random.randint(3,15,size=N),
                  'weight': np.random.uniform(0,4,size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight']
                 )
fruit_cat = df['fruit'].astype('category')
print(fruit_cat,'\n')
print(type(fruit_cat.values))
c=fruit_cat.values
print(c.categories)
print(c.codes)
df['fruit'] = df['fruit'].astype('category')
print(df['fruit'], '\n')
#自己构建category
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
#从2个lists构建：
categories = ['foo', 'bar', 'baz']
codes=[0,1,2,0,0,1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
print(my_cats_2, '\n')
#对category排序
my_cats_2 = pd.Categorical.from_codes(codes, categories, ordered=True)
my_cats_2.as_ordered()

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object 

##################Categorical Type in pandas######################
0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange] 

<class 'pandas.core.arrays.categorical.Categorical'>
Index(['apple', 'orange'], dtype='object')
[0 1 0 0 0 1 0 0]
0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange] 

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz] 



In [27]:
print('##################Computations with Categoricals######################')
#pandas.cut对一列数分成不同区间的category用法
np.random.seed(12345)
draws = np.random.randn(1000)
bins = pd.qcut(draws,4)
print(bins, '\n')
#给上面例子每个quantile一个名字可以使得结果更好理解
bins = pd.qcut(draws, 4, labels=['Q1','Q2','Q3','Q4'])
print(bins, '\n')
print(bins.categories, '\n')
print(bins.codes[:10],'\n')
bins = pd.Series(bins, name='quartile')
print(bins, '\n')
results = pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index()
print(pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index())
print(results['quartile'], '\n')

##################Computations with Categoricals######################
[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]] 

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4] 

Index(['Q1', 'Q2', 'Q3', 'Q4'], dtype='object') 

[1 2 1 1 3 3 2 2 3 3] 

0      Q2
1      Q3
2      Q2
3      Q2
4      Q4
       ..
995    Q3
996    Q2
997    Q1
998    Q3
999    Q4
Name: quartile, Length: 1000, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4] 

  quartile  count       min       max
0       Q1    250 -2.949343 -0.685484
1       Q2    250 -0.683066 -0.010115
2       Q3    250 -0.010032  0.628894
3       Q4    250  0.634238  3.927528
0    Q1
1    Q2
2    Q3
3   

In [31]:
print('##################Better performance with categoricals######################')
#如果对特定数据集做大量研究，把数据转为categorical会提高执行效率。a categorical version用的内存更少
N=10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo','bar','baz','qux'] * (N//4))
categories = labels.astype('category')
print(labels.memory_usage())
print(categories.memory_usage(), '\n')
%time _ = labels.astype('category')

##################Better performance with categoricals######################
80000128
10000320 

CPU times: user 318 ms, sys: 18.2 ms, total: 336 ms
Wall time: 335 ms


In [43]:
print('##################Categorical Methods######################')
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
print(cat_s)
print(cat_s.cat.codes, '\n')
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
print(cat_s2, '\n')
print(cat_s.value_counts(), '\n')
print(cat_s2.value_counts(),'\n')
#用category筛选需要的categories
cat_s3 = cat_s[cat_s.isin(['a','b'])]
print(cat_s3, '\n')
#使用remove_unused_categories可以去除不使用的那些categories，比如这里除了a和b之外的cacategories.
cat_s3.cat.remove_unsed_categories()

##################Categorical Methods######################
0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]
0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8 

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e] 

d    2
c    2
b    2
a    2
dtype: int64 

d    2
c    2
b    2
a    2
e    0
dtype: int64 

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d] 



In [46]:
print('##################Creating dummy variables for modeling######################')
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
print(pd.get_dummies(cat_s), '\n')

##################Creating dummy variables for modeling######################
   a  b  c  d
0  1  0  0  0
1  0  1  0  0
2  0  0  1  0
3  0  0  0  1
4  1  0  0  0
5  0  1  0  0
6  0  0  1  0
7  0  0  0  1 



#### 2 Advanced GroupBy Use

In [57]:
print('##################Group Transforms and “Unwrapped” GroupBys######################')
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                    'value': np.arange(12.)})
g = df.groupby('key').value
print(g.mean())
#同样输出mean，但是返回每个样本
g.transform(lambda x: x.mean())
#对于build-in aggregation公式，可以直接用
g.transform('mean')
g.transform(lambda x: x *2)
print(g.transform(lambda x: x.rank(ascending=False)), '\n')
def normalize(x):
    return (x-x.mean())/x.std()
print(g.transform(normalize), '\n')
print(g.apply(normalize),'\n')
normalized = (df['value'] - g.transform('mean'))/g.transform('std')
print(normalized, '\n')

##################Group Transforms and “Unwrapped” GroupBys######################
key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64
0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64 

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64 

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64 

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64 



In [65]:
print('##################Grouped Time Resampling######################')
N=15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times, 'value':np.arange(N)})
print(df,'\n')
print(df.set_index('time').resample('5min').count(), '\n')

#含有多个time series   ???没有pd.TimeGrouper()函数
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
time_key = pd.TimeGrouper('5min')
resampled = df2.set_index('time').groupby(['key', time_key]).sum()
print(resampled,'\n')

##################Grouped Time Resampling######################
                  time  value
0  2017-05-20 00:00:00      0
1  2017-05-20 00:01:00      1
2  2017-05-20 00:02:00      2
3  2017-05-20 00:03:00      3
4  2017-05-20 00:04:00      4
5  2017-05-20 00:05:00      5
6  2017-05-20 00:06:00      6
7  2017-05-20 00:07:00      7
8  2017-05-20 00:08:00      8
9  2017-05-20 00:09:00      9
10 2017-05-20 00:10:00     10
11 2017-05-20 00:11:00     11
12 2017-05-20 00:12:00     12
13 2017-05-20 00:13:00     13
14 2017-05-20 00:14:00     14 

                     value
time                      
2017-05-20 00:00:00      5
2017-05-20 00:05:00      5
2017-05-20 00:10:00      5 



AttributeError: module 'pandas' has no attribute 'TimeGrouper'

#### 3 Techniques for Method Chaining

In [ ]:
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()

#对于大量临时变量，DataFrame.assign 是高效的column assignment
df2 = df.copy()
df2['k'] = v
#use assign
df2 = df.assign(k=v)
#直接修改比assign执行快，但是assign使得method chaining更容易
result = df2.assign(col1_demeaned=df2.col1 - df2.col1.mean()).groupby('key').col1_demeaned.std()


#callables
df = (load_data() [lambda x:x['col2'] < 0])


result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean()) .groupby('key')
          .col1_demeaned.std())



In [ ]:
print('##################The pipe Method######################')
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
#pipe可以接受并返回series和dataframe objects，这里f(df)和df.pipe(f)一样效果
result = (df.pipe(f,arg1=v1)
         .pipe(g, v2, arg3=v3)
         .pipe(h, b, arg4=v4))

g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')

def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = result[c] - g.transform('mean')
    return result

result = df[df.col1 < 0].pipe(group_demean, ['key1', 'key2'], ['col1'])
